In [78]:
import numpy as np
import cvxpy as cvx
import random
import pandas as pd


In [79]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A.csv")
pi = pd.read_csv("pi.csv")

A = np.array(A)
pi = np.array(pi)
pi = pi.T
b = np.array([1000 for _ in range(m)])  


OffLine

In [80]:
x = cvx.Variable(n)
objective = cvx.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= 1]
prob = cvx.Problem(objective, constraints)

prob.solve(verbose = True)
# x.value

print(x.value)
opv1 = prob.value
print(opv1)

x = x.value

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Apr 03 12:57:56 AM: Your problem has 10000 variables, 3 constraints, and 0 parameters.
(CVXPY) Apr 03 12:57:56 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 03 12:57:56 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 03 12:57:56 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 03 12:57:56 AM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 03 12:57:56 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

K = 50

In [81]:
k = 50
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T                       # A: m*k
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))     # pi: k*1
bNow = b*(float(k/n))
bNow = bNow.T                       # b: m*1

temp = m+k                          # 前m个是y, 后k个是p
y = cvx.Variable(temp)
e = np.ones(k)
objective = cvx.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ y[m:])>= piNow, y >= 0]
prob = cvx.Problem(objective, constraints)

prob.solve(verbose = True)
# print("==============")
# print(prob.value)
# print(y.value.shape)
Y = y[:m].value
# print(P)

AT = A.T
# print("----------")
# print(AT[0].dot(P))

xk50 = []
for i in range(k):
    cri = (AT[i].dot(Y))
    # print(cri)
    # print(type(cri))
    if(pi[0][i] > cri):
        xk50.append(1)
    else:
        xk50.append(0)

# xk50
xk50 = np.array(xk50)

xk50

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Apr 03 12:57:56 AM: Your problem has 60 variables, 2 constraints, and 0 parameters.
(CVXPY) Apr 03 12:57:56 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 03 12:57:56 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 03 12:57:56 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 03 12:57:56 AM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 03 12:57:56 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0])

In [82]:
opv2 = (piNow.dot(xk50))
rateK50 = opv2/opv1
# print(opv2)
rateK50

0.004780187071991184

k100

In [83]:
k = 100
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))
bNow = b*(float(k/n))
bNow = bNow.T

y = cvx.Variable((m+k))
e = np.ones(k)
objective = cvx.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cvx.Problem(objective, constraints)

prob.solve(verbose = True)
Y = y[:m].value

AT = A.T
# print("----------")
# print(AT[0])

xk100 = []
for i in range(k):
    cri = (AT[i].dot(Y))
    # print(cri)
    if(pi[0][i] > cri):
        xk100.append(1)
    else:
        xk100.append(0)

# xk100
xk100 = np.array(xk100)


                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Apr 03 12:57:57 AM: Your problem has 110 variables, 2 constraints, and 0 parameters.
(CVXPY) Apr 03 12:57:57 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 03 12:57:57 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 03 12:57:57 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 03 12:57:57 AM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 03 12:57:57 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

In [84]:
# opv1 = (pi.dot(x))[0]
opv3 = (piNow.dot(xk100))
# print(opv1)
# print(type(opv1))
rateK100 = opv3/opv1

rateK100

0.00954520760627491

k200

In [85]:
k = 200
steps = n/k
Anow = A[:, 0:k]
Anow = Anow.T
# print(Anow.shape)
piNow = (pi.T)[0:k]
piNow = np.reshape(piNow, (k,))
bNow = b*(float(k/n))
bNow = bNow.T


y = cvx.Variable((m+k))
e = np.ones(k)
objective = cvx.Minimize(bNow@(y[:m]) + e@(y[m:]))
constraints = [(Anow@(y[:m])+ (y[m:]))>= piNow, y >= 0]
prob = cvx.Problem(objective, constraints)

prob.solve(verbose = True)
Y = y[:m].value

AT = A.T
# print("----------")
# print(AT[0])

xk200 = []
for i in range(k):
    cri = (AT[i].dot(Y))
    # print(cri)
    if(pi[0][i] > cri):
        xk200.append(1)
    else:
        xk200.append(0)

# xk200
xk200 = np.array(xk200)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Apr 03 12:57:57 AM: Your problem has 210 variables, 2 constraints, and 0 parameters.
(CVXPY) Apr 03 12:57:57 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 03 12:57:57 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 03 12:57:57 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 03 12:57:57 AM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 03 12:57:57 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

In [86]:
# opv1 = (pi.dot(x))[0]
opv4 = (piNow.dot(xk200))
# print(opv1)
# print(type(opv1))
rateK200 = opv4/opv1
# print(xk200)
rateK200

0.019527855316936297